In [21]:
import json
import os
from omero.rtypes import rstring
from omero.gateway import BlitzGateway
from biohack_utils.config_utils import flatten, unflatten, download, upload
from biohack_utils.ConfigSchema import OMEWrapper
from biohack_utils.ConfigSchema import CollectionNode
from biohack_utils.util import connect_to_omero
import dotenv
import pprint


In [22]:
def connect_to_omero():
    dotenv.load_dotenv()
    USERNAME = os.getenv("OMERO_USERNAME")
    PASSWORD = os.getenv("OMERO_PASSWORD")
    HOST = "omero-training.gerbi-gmb.de"
    PORT = 4064  # Default OMERO port

    conn = BlitzGateway(USERNAME, PASSWORD, host=HOST, port=PORT)
    conn.connect()

    if conn.isConnected():
        print("Connected to OMERO")
    else:
        print("Failed to connect")
        exit(1)

    return conn


In [23]:
dotenv.load_dotenv()
    # Read the json file
json_file_path = "example2.json"
    
with open(json_file_path) as f:
    d = json.load(f)

In [24]:
wrapper = OMEWrapper.model_validate(d)

In [25]:
data_dict = wrapper.model_dump()

# 3. Export with Indentation 💾
file_path = "output_from_wrapper.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(data_dict, f, indent=2)

In [26]:
flat = flatten(wrapper)
pprint.pprint(flat)

[{'category': 'intensities',
  'description': 'Original nuclei channel',
  'omero:image_id': 35478,
  'origin': 'raw',
  'path': 'nuclei_raw'},
 {'category': 'annotations',
  'description': 'Nuclei segmentation masks',
  'omero:image_id': 35485,
  'origin': 'masks',
  'path': 'nuclei_segmentation',
  'source': 'nuclei_raw'},
 {'category': 'annotations',
  'description': 'Cell segmentation masks',
  'omero:image_id': 35489,
  'origin': 'masks',
  'path': 'cell_segmentation',
  'source': 'nuclei_raw'}]


In [27]:
# Unflatten example
unflattened = unflatten(flat, "cell_analysis").model_dump()
# 3. Export with Indentation 💾
file_path = "output_from_after_unwrapping.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(unflattened, f, indent=2)

In [28]:
conn = connect_to_omero()

Connected to OMERO


In [36]:
coll_id = upload(conn,wrapper)

In [37]:
schema = download(conn, coll_id)


Processing image 35478
  Found 1 node annotations via query
  ✓ Match found!

Processing image 35485
  Found 1 node annotations via query
  ✓ Match found!

Processing image 35489
  Found 1 node annotations via query
  ✓ Match found!

Total flat records collected: 3


In [32]:
schema

# 3. Export with Indentation 💾
file_path = "output_from_omero.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(data_dict, f, indent=2)

In [35]:
# Get the image  
image = conn.getObject("Image", 35485)  
  
# Delete all map annotations (by namespace)  
image.removeAnnotations(ns="ome/collection")  # Specify namespace or None for all
image.removeAnnotations(ns="ome/collection/nodes")  # Specify namespace or None for all

 